# Silhouette

This notebook creates a new scenario with the silhouette of the person in the video. The silhouette is created by using a pre-trained model from detectron2. The model is used to segment the person in the video and the silhouette is created by multiplying the frame with the mask of the person.

## Setup detectron2 and load the model

In [ ]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
# model = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
model = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
config_path = model_zoo.get_config_file(model)

cfg.merge_from_file(config_path)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

In [ ]:
# Get the mask of the person
person_class = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes.index("person")

In [ ]:
predictor = DefaultPredictor(cfg)

## Create a new scenario

In [ ]:
def get_person_mask(frames):
    """This function returns the segmentation masks of the person in the frames"""
    masks = []
    for frame in frames:
        outputs = predictor(frame)
        prediction_masks = outputs["instances"].pred_masks
        prediction_classes = outputs["instances"].pred_classes
        person_mask = prediction_masks[prediction_classes == person_class][0].cpu().numpy()
        masks.append(person_mask)
    return masks

In [ ]:
import cv2
import numpy as np


def store_video_with_mask(frames, masks, filename, fps):
    shape = frames[0].shape

    fourcc = cv2.VideoWriter_fourcc(*'FFV1')
    out = cv2.VideoWriter(filename, fourcc, fps, (shape[1], shape[0]))

    for idx, mask in enumerate(masks):
        frame = np.uint8(frames[idx] * mask[:, :, None])
        out.write(frame)

    out.release()

In [ ]:
import os
import subprocess
from respiration.dataset import VitalCamSet
from tqdm.auto import tqdm

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

for (subject, setting) in tqdm(scenarios):
    print(f"subject: {subject}")

    destination = os.path.join(dataset.data_path, subject, '301_silhouette')

    # Copy the unisens data to the new scenario
    source = os.path.join(dataset.data_path, subject, setting, 'synced_Logitech HD Pro Webcam C920')
    os.makedirs(destination, exist_ok=True)
    subprocess.run(["cp", "-r", source, destination])

    frames, meta = dataset.get_video_rgb(subject, setting)
    masks = get_person_mask(frames)
    video_path = os.path.join(destination, 'Logitech HD Pro Webcam C920.avi')
    store_video_with_mask(frames, masks, video_path, meta.fps)